# Generate data for all figures and export as CSV files for plotting in TikZ

### Subfunction to generate the grids to evaluate the models over

In [124]:
function create_grids(ca, V)
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur = Array(Any, (1*length(V),))
    x_grid_cur_dense = Array(Any, (1*length(Vdense),))
    iterind = 0;
    for j1 in product([ca], V)
        iterind += 1;
        x_grid_cur[iterind] = collect(j1)
    end
    iterind = 0;
    for j1 in product([ca], Vdense)
        iterind += 1;
        x_grid_cur_dense[iterind] = collect(j1)
    end
        
    return x_grid_cur, x_grid_cur_dense
end

create_grids (generic function with 1 method)

# Figure 1

In [125]:
# Figure 1 A - Drawn in TikZ

In [126]:
# Figure 1 B and C

# Load the BK functions, the x_grid(Ca, V) that we evaluate over and the initial condition phi0
include("../BK_functions/bk_setup_script.jl"); 

Vdense = minimum(V):3e-3:maximum(V) # voltage sampled every 3 millivolts for smooth lines

for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur, x_grid_cur_dense = create_grids(ca, V);
    
    # Get the function values for original model
    tmp = BK_simulator(phi0, hcat(x_grid_cur...), model_id=10)
    writecsv("CSV/figure_1b_panel1_ca$(ca_str)um.csv", hcat(V[:]*1000, tmp[:]))
    
    # Save also the log10 values
    writecsv("CSV/figure_1c_panel1_ca$(ca_str)um.csv", hcat(V[:]*1000, log10(tmp[:])))
    
    # Densely sampled
    tmp = BK_simulator(phi0, hcat(x_grid_cur_dense...), model_id=10)
    writecsv("CSV/figure_1b_panel2_ca$(ca_str)um.csv", hcat(Vdense[:]*1000, tmp[:]))
    writecsv("CSV/figure_1c_panel2_ca$(ca_str)um.csv", hcat(Vdense[:]*1000, log10(tmp[:])))
end

# Figure 2

In [127]:
# Figure 2 A
tmp = load("saves/figure_A1_hess_"*"8"*".jld") # Example noisy fit with diverging parameters
phi1 = tmp["phi1_orig"] # Get the fitted parameter vector

# Output the simulated data point markers in a CSV format with: 
# first column - voltage (mV), second column - Popen, third column error top, fourth column

# Also output the example fit with just first col - voltage (mV), second col - Popen
# but with voltage sampled more densely to make it a continous line

for ca in Ca  
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur, x_grid_cur_dense = create_grids(ca, V);
    
    # Get the function values for original model
    tmp = BK_simulator(phi0, hcat(x_grid_cur...), model_id=0)
    errbar = tmp.*0.1 # Generate the error bar for every data point.
    writecsv("CSV/figure_2a_panel1_ca$(ca_str)um.csv", hcat(V[:].*1000, tmp[:], errbar[:], errbar[:]))
    
    # Get the function values for the fit to noisy data (densely sampled)
    tmp = BK_simulator(phi1, hcat(x_grid_cur_dense...), model_id=0)
    writecsv("CSV/figure_2a_panel2_ca$(ca_str)um.csv", hcat(Vdense[:].*1000, tmp[:]))
end

In [130]:
# Figure 2 B
# Output the original and the fitted parameter values (first and second column of CSV respectively)
writecsv("CSV/figure_2b_panel1.csv", [phi0 phi1])

In [129]:
phi1

8-element Array{Float64,1}:
 7.69353e-37
 8.75658e-33
 0.0662813  
 0.586839   
 2.74314e-5 
 1.72186e8  
 2.39839e9  
 4.54419e-8 

# Figure 3

In [19]:
# Figure 3 A
# Table with theoretical lower bound on parameter uncertainties

# REF NEEDED
# CALCULATION NEEDED
# DERIVATION NEEDED

In [20]:
# Figure 3 B - Drawn in TikZ

In [21]:
# Figure 3 C
using JLD
outp = load("saves/figure3_smallVnorm.jld", "outp");
outp_log10 = load("saves/figure4.jld", "outp_log10");

writecsv("CSV/figure_3c_panel1_orig.csv", hcat(1:size(outp[3][1,1],1), log(outp[3][1,1])))
writecsv("CSV/figure_3c_panel1_log10.csv", hcat(1:size(outp_log10[3][1,1],1), log(outp_log10[3][1,1])))

# Figure 4

In [22]:
# Figure 4 A
# Cost for original space model
# writecsv("CSV/figure_4a_panel1.csv", hcat(0:size(outp[1],1), vcat(outp[2][1,1], outp[2][:,2])))
writecsv("CSV/figure_4a_panel1.csv", hcat(0:5, vcat(outp[2][1,1], outp[2][1:5,2])))

In [23]:
# Figure 4 B
# Left - drawn in TikZ
# Right - Data: Reuse same data points as in Figure 1 B
# Right - Fit:
for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur, x_grid_cur_dense = create_grids(ca, V);
    
    # Get the function values for the fit to noisy data (densely sampled)
    tmp = BK_simulator(exp(outp[1][4,1]), hcat(x_grid_cur_dense...), model_id=13)
    writecsv("CSV/figure_4b_panel2_ca$(ca_str)um.csv", hcat(Vdense[:].*1000, tmp[:]))
end

In [24]:
# Figure 4 C
# Left - drawn in TikZ
# Right - Data: Reuse same data points as in Figure 1 B
# Right - Fit:
for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur, x_grid_cur_dense = create_grids(ca, V);
    
    # Get the function values for the fit to noisy data (densely sampled)
    tmp = BK_simulator(exp(outp[1][5,1]), hcat(x_grid_cur_dense...), model_id=14)
    writecsv("CSV/figure_4c_panel2_ca$(ca_str)um.csv", hcat(Vdense[:].*1000, tmp[:]))
end

# Figure 5 / Figure A1

Show the MBAM reduction steps (in Figure 5 schematically, in Figure A1 with more details)

* Panel n - nth reduction containing each parameter curve

In [25]:
# Naming the original and reduced parameter vectors for each model reduction step

param_names = Array(Array{AbstractString},size(outp[1],1));
param_names[1] = ["L0", "zL", "J0", "zJ", "Kd", "C", "D", "E"]
param_names[2] = ["L0", "J0", "zJ", "Kd", "C", "D", "E"]
param_names[3] = ["J0", "zJ", "Kd", "LC","LD", "EL"];
param_names[4] = ["J0", "zJ", "Kd", "CE","LD"];
param_names[5] = ["LDJ", "zJ", "Kd", "CE"];
param_names[6] = ["LDJ", "zJ", "CEKd"];

In [26]:
for model_red = 1:size(outp[1],1)
    # Get the log param values during the reduction
    phi_red_time = outp[5][model_red]
    phi_red_vals = hcat(outp[6][model_red]...)
    phi_red_vals = phi_red_vals[1:div(size(phi_red_vals,1),2),:]
    
    # Find the converged time (the one after no parameters change)
    cutoff = 0
    for i1 = 2:size(phi_red_vals,2)
        if all(phi_red_vals[:,i1].==phi_red_vals[:,i1-1])
            cutoff = i1
            break
        end
    end
    
    # Produce the geodesic time - log(value) for each parameter for each reduction step
    for param_num in 1:size(phi_red_vals,1)
        writecsv("CSV/figure_5"*string("abcdef"[model_red])*"_panel1_param_$(param_names[model_red][param_num]).csv", 
            hcat(phi_red_time[1:cutoff], phi_red_vals[param_num,1:cutoff][:]))
    end
end

# Figure 6

In [27]:
# Figure 6 A
# Cost for log space model
writecsv("CSV/figure_6a_panel1.csv", hcat(0:4, vcat(outp_log10[2][1,1], outp_log10[2][1:4,2])))

In [28]:
# Figure 6 B
# Left - drawn in TikZ
# Right - Data: Reuse same data points as in Figure 1 C
# Right - Fit:
for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur, x_grid_cur_dense = create_grids(ca, V);
    
    # Get the function values for the fit to noisy data (densely sampled)
    tmp = BK_simulator(exp(outp_log10[1][2,1]), hcat(x_grid_cur_dense...), model_id=52)
    writecsv("CSV/figure_6b_panel2_ca$(ca_str)um.csv", hcat(Vdense[:].*1000, tmp[:]))
end

In [29]:
# Figure 6 C
# Left - drawn in TikZ
# Right - Data: Reuse same data points as in Figure 1 C
# Right - Fit:
for ca in Ca
    ca_str = @sprintf("%1.1f", ca*1e6)
    
    # Generate grids to evaluate over
    x_grid_cur, x_grid_cur_dense = create_grids(ca, V);
    
    # Get the function values for the fit to noisy data (densely sampled)
    tmp = BK_simulator(exp(outp_log10[1][3,1]), hcat(x_grid_cur_dense...), model_id=53)
    writecsv("CSV/figure_6c_panel2_ca$(ca_str)um.csv", hcat(Vdense[:].*1000, tmp[:]))
end

# Figure 7

In [30]:
# RELATIVE ERRORS TO BE RECALCULATED AND DERIVED

# Figure 8

In [110]:
include("../BK_functions/bk_setup_script.jl")

# Original and Simulation parameters (fit the orig assay, see Figure 2)
phi1_orig = load("results/noisy_fits_to_diverging_1.jld", "metadata")["phi1_orig"]
[phi0 phi1_orig]

# Load log fits to noisy data simulated with parameters above (both original and diverging)
using JLD
all_vars = Dict();
orig_vars = Dict();
keys = 52
var = "phi1_log"
orig_var = "phi1_log"
n=0
for i in [1:51; 151:199]
    n+=1;
    tmp_div = load("results/noisy_fits_to_diverging_"*string(i)*".jld", "noisy_fits")
    tmp_orig = load("results/noisy_fits_"*string(n)*".jld", "noisy_fits")
    for key in keys
        if !haskey(all_vars, key)
            all_vars[key] = Array(Any, 100);
        end
        if !haskey(orig_vars, key)
            orig_vars[key] = Array(Any, 100);
        end
        all_vars[key][n] = tmp_div[key][var]
        orig_vars[key][n] = tmp_orig[key][var]
    end
end

### Compensation between C and E

In [111]:
# Matrix of all fits for C and E (base parameters, reduced log fit)
base_fits = log10(hcat(orig_vars[keys][1:n]...)[[5,7],:])

# Matrix of all fits for C and E (diverging parameters, reduced log fit)
diverg_fits = log10(hcat(all_vars[keys][1:n]...)[[5,7],:])

# True base params and diverging params for C and E
true_params = log10(hcat(phi0[[6,8]], phi1_orig[[6,8]]))



2x2 Array{Float64,2}:
 0.789581   8.236  
 0.30103   -7.34254

In [112]:
# Defining a coloring split for "diverging all fits" based on whether the fitted 
# E is above or below the true diverging parameter

cloud_membership = (diverg_fits[1,:].>mean(diverg_fits[1,:]));

In [113]:
sum(cloud_membership)

47

In [114]:
# Export the appropriate CSVs

avg_func(x) = mean(x)

# Figure A, changes in log10(C)
# True base param vs true diverging param
writecsv("CSV/figure_8a_panel1.csv", hcat([0; 1], true_params[1,:][:]))
# Average fits
writecsv("CSV/figure_8a_panel2.csv", hcat([0; 1], vcat(avg_func(base_fits[1,:]), avg_func(diverg_fits[1,:]))))
# All fits - split by relationship of fit C to mean of C
# Fits of base
writecsv("CSV/figure_8a_panel3.csv", hcat(randn(100,1)*0.1, base_fits[1,:]'))
writecsv("CSV/figure_8a_panel4.csv", hcat(randn(sum(cloud_membership),1)*0.1+1, diverg_fits[1, cloud_membership]'))
writecsv("CSV/figure_8a_panel5.csv", hcat(randn(sum(!cloud_membership),1)*0.1+1, diverg_fits[1, !cloud_membership]'))


# Figure B, changes in log10(E)
# True base param vs true diverging param
writecsv("CSV/figure_8b_panel1.csv", hcat([0; 1], true_params[2,:][:]))
# Average fits
writecsv("CSV/figure_8b_panel2.csv", hcat([0; 1], vcat(avg_func(base_fits[2,:]), avg_func(diverg_fits[2,:]))))
# All fits - split by relationship of fit C to mean of C
# Fits of base
writecsv("CSV/figure_8b_panel3.csv", hcat(randn(100,1)*0.1, base_fits[2,:]'))
writecsv("CSV/figure_8b_panel4.csv", hcat(randn(sum(cloud_membership),1)*0.1+1, diverg_fits[2, cloud_membership]'))
writecsv("CSV/figure_8b_panel5.csv", hcat(randn(sum(!cloud_membership),1)*0.1+1, diverg_fits[2, !cloud_membership]'))


# Figure C, changes in log10(C*E)
# True base param vs true diverging param
writecsv("CSV/figure_8c_panel1.csv", hcat([0; 1], sum(true_params,1)[:]))
# Average fits
writecsv("CSV/figure_8c_panel2.csv", hcat([0; 1], vcat(avg_func(sum(base_fits,1)), avg_func(sum(diverg_fits,1)))))
# All fits - split by relationship of fit C to mean of C
# Fits of base
writecsv("CSV/figure_8c_panel3.csv", hcat(randn(100,1)*0.1, sum(base_fits,1)'))
writecsv("CSV/figure_8c_panel4.csv", hcat(randn(sum(cloud_membership),1)*0.1+1, sum(diverg_fits[:, cloud_membership],1)'))
writecsv("CSV/figure_8c_panel5.csv", hcat(randn(sum(!cloud_membership),1)*0.1+1, sum(diverg_fits[:, !cloud_membership],1)'))

### Compensation between L0 and D

In [115]:
# Matrix of all fits for L0 and D (base parameters, reduced log fit)
base_fits = log10(hcat(orig_vars[keys][1:n]...)[[1,6],:])

# Matrix of all fits for L0 and D (diverging parameters, reduced log fit)
diverg_fits = log10(hcat(all_vars[keys][1:n]...)[[1,6],:])

# True base params and diverging params for L0 and D
true_params = log10(hcat(phi0[[1,7]], phi1_orig[[1,7]]))

2x2 Array{Float64,2}:
 -5.65758  -36.1139 
  1.48287    9.37992

In [116]:
# Export the appropriate CSVs

# Figure D, changes in log10(L0)
# True base param vs true diverging param
writecsv("CSV/figure_8d_panel1.csv", hcat([0; 1], true_params[1,:][:]))
# Average fits
writecsv("CSV/figure_8d_panel2.csv", hcat([0; 1], vcat(avg_func(base_fits[1,:]), avg_func(diverg_fits[1,:]))))
# All fits - split by relationship of fit C to mean of C
# Fits of base
writecsv("CSV/figure_8d_panel3.csv", hcat(randn(100,1)*0.1, base_fits[1,:]'))
writecsv("CSV/figure_8d_panel4.csv", hcat(randn(sum(cloud_membership),1)*0.1+1, diverg_fits[1, cloud_membership]'))
writecsv("CSV/figure_8d_panel5.csv", hcat(randn(sum(!cloud_membership),1)*0.1+1, diverg_fits[1, !cloud_membership]'))



# Figure E, changes in log10(D)
# True base param vs true diverging param
writecsv("CSV/figure_8e_panel1.csv", hcat([0; 1], true_params[2,:][:]))
# Average fits
writecsv("CSV/figure_8e_panel2.csv", hcat([0; 1], vcat(avg_func(base_fits[2,:]), avg_func(diverg_fits[2,:]))))
# All fits - split by relationship of fit C to mean of C
# Fits of base
writecsv("CSV/figure_8e_panel3.csv", hcat(randn(100,1)*0.1, base_fits[2,:]'))
writecsv("CSV/figure_8e_panel4.csv", hcat(randn(sum(cloud_membership),1)*0.1+1, diverg_fits[2, cloud_membership]'))
writecsv("CSV/figure_8e_panel5.csv", hcat(randn(sum(!cloud_membership),1)*0.1+1, diverg_fits[2, !cloud_membership]'))


# Figure F, changes in log10(L0^-0.25 * D)
# True base param vs true diverging param
writecsv("CSV/figure_8f_panel1.csv", hcat([0; 1], sum(true_params.*[0.25,1],1)[:]))
# Average fits
writecsv("CSV/figure_8f_panel2.csv", hcat([0; 1], vcat(avg_func(sum(base_fits.*[0.25,1],1)), avg_func(sum(diverg_fits.*[0.25,1],1)))))
# All fits - split by relationship of fit C to mean of C
# Fits of base
writecsv("CSV/figure_8f_panel3.csv", hcat(randn(100,1)*0.1, sum(base_fits.*[0.25,1],1)'))
writecsv("CSV/figure_8f_panel4.csv", hcat(randn(sum(cloud_membership),1)*0.1+1, sum(diverg_fits[:, cloud_membership].*[0.25,1],1)'))
writecsv("CSV/figure_8f_panel5.csv", hcat(randn(sum(!cloud_membership),1)*0.1+1, sum(diverg_fits[:, !cloud_membership].*[0.25,1],1)'))


### Non-compensating parameters - J0, zJ, Kd

In [117]:
# Matrix of all fits for J0, zJ, Kd (base parameters, reduced log fit)
base_fits = log10(hcat(orig_vars[keys][1:n]...)[[2,3,4],:])

# Matrix of all fits for J0, zJ, Kd (diverging parameters, reduced log fit)
diverg_fits = log10(hcat(all_vars[keys][1:n]...)[[2,3,4],:])

# True base params and diverging params for J0, zJ, Kd
true_params = log10(hcat(phi0[[3,4,5]], phi1_orig[[3,4,5]]))

3x2 Array{Float64,2}:
 -0.988853  -1.17861 
 -0.236572  -0.231481
 -4.40894   -4.56175 

In [121]:
[phi0 phi1_orig]

8x2 Array{Float64,2}:
  2.2e-6  7.69353e-37
  0.42    8.75658e-33
  0.1026  0.0662813  
  0.58    0.586839   
  3.9e-5  2.74314e-5 
  6.16    1.72186e8  
 30.4     2.39839e9  
  2.0     4.54419e-8 

In [119]:
# Export the appropriate CSVs

# Figure G, changes in log10(J0)
# True base param vs true diverging param
writecsv("CSV/figure_8g_panel1.csv", hcat([0; 1], true_params[1,:][:]))
# Average fits
writecsv("CSV/figure_8g_panel2.csv", hcat([0; 1], vcat(avg_func(base_fits[1,:]), avg_func(diverg_fits[1,:]))))
# All fits - split by relationship of fit C to mean of C
# Fits of base
writecsv("CSV/figure_8g_panel3.csv", hcat(randn(100,1)*0.1, base_fits[1,:]'))
writecsv("CSV/figure_8g_panel4.csv", hcat(randn(sum(cloud_membership),1)*0.1+1, diverg_fits[1, cloud_membership]'))
writecsv("CSV/figure_8g_panel5.csv", hcat(randn(sum(!cloud_membership),1)*0.1+1, diverg_fits[1, !cloud_membership]'))



# Figure H, changes in log10(zJ)
# True base param vs true diverging param
writecsv("CSV/figure_8h_panel1.csv", hcat([0; 1], true_params[2,:][:]))
# Average fits
writecsv("CSV/figure_8h_panel2.csv", hcat([0; 1], vcat(avg_func(base_fits[2,:]), avg_func(diverg_fits[2,:]))))
# All fits - split by relationship of fit C to mean of C
# Fits of base
writecsv("CSV/figure_8h_panel3.csv", hcat(randn(100,1)*0.1, base_fits[2,:]'))
writecsv("CSV/figure_8h_panel4.csv", hcat(randn(sum(cloud_membership),1)*0.1+1, diverg_fits[2, cloud_membership]'))
writecsv("CSV/figure_8h_panel5.csv", hcat(randn(sum(!cloud_membership),1)*0.1+1, diverg_fits[2, !cloud_membership]'))

# Figure I, changes in log10(Kd)
# True base param vs true diverging param
writecsv("CSV/figure_8i_panel1.csv", hcat([0; 1], true_params[3,:][:]))
# Average fits
writecsv("CSV/figure_8i_panel2.csv", hcat([0; 1], vcat(avg_func(base_fits[3,:]), avg_func(diverg_fits[3,:]))))
# All fits - split by relationship of fit C to mean of C
# Fits of base
writecsv("CSV/figure_8i_panel3.csv", hcat(randn(100,1)*0.1, base_fits[3,:]'))
writecsv("CSV/figure_8i_panel4.csv", hcat(randn(sum(cloud_membership),1)*0.1+1, diverg_fits[3, cloud_membership]'))
writecsv("CSV/figure_8i_panel5.csv", hcat(randn(sum(!cloud_membership),1)*0.1+1, diverg_fits[3, !cloud_membership]'))
